# 卷积

In [1]:
import torch
from torch import nn

# 从全连接层到卷积

全连接的网络结构，比较适合处理一维的数据，每一维是某一个特征，特征与特征之间也没有考虑相关性。

但对于像是图片这样的数据源，全连接网络就很吃力了，对于一个`800*600`分辨率的图片来说，总的像素数量已经达到了几十万，如果考虑到RGB多通道，就上百万了。那么第一个隐藏层的参数，很容易就上亿了。这样的模型很难训练。一是对训练机器的存储和算力要求都很高，其次是参数太多，就需要更多的数据，不然很容易过拟合。

对于图像识别这样的场景，我们一般需要考虑到图像本身的结构。

## 设计原则

- 平移不变性，神经网络底层（靠近输入的层）对图像上不同位置上的相同内容应该输出类似的响应，比如不能因为一个猫出现在图像的左下角，而不是中心，就识别不出来。
- 局部性，神经网络的底层应该只探索输入图像的局部区域，而不考虑图像远处区域的内容，提取出来局部特征，再通过高层的层来对底层特征进行组合。

## 公式变换

我们来考察考虑到上面2个设计原则后，我们如何改进原来的全连接层的设计。在全连接层中，假设隐藏层也看成一个2维的结构H，它的每个隐藏神经元的输入，表示为：
$$H_{i,j} = \sum_k\sum_l W_{i,j,k,l}X_{k,l} + B_{i,j}$$

其中$H_{i,j}$表示隐藏层位置$(i,j)$上的输出，该位置相关的参数为$W_{i,j}$，它是一个和输入$X$具有相同shape的参数。$H_{i,j}$就是整个$X$上所有位置的值与$W_{i,j}$的加权求和。

考虑上平移不变化，权重不应该在$H$的不同位置上，表现不同。所以公式，就变成了。

$$H_{i,j} = \sum_k\sum_l W_{k,l}X_{k,l} + b$$

考虑上局部性，对于$H$的位置$(i,j)$，不需要考虑$X$上的所有像素点，只需要考虑，对应位置的一个邻域的像素点。公式就变成了。

$$H_{i,j} = \sum_{a=-\Delta}^{\Delta}\sum_{b=-\Delta}^{\Delta} W_{a,b}X_{i + a, j + b} + b$$

# 考虑上通道

由于输入的图像，可能彩色的，所以它其实是有3个二维矩阵组成的3维的结构，也就是多了一个通道的维度。
$$C\times H \times W$$

这样，如果我们权重相作用于一个带通道的输入上时，我们的权重就也由原来的二维$W_{a,b}$变成了，三维的$W_{c,a,b}$。

另外如果我们的隐藏层，如果想有更强的表达能力，那就需要有多个通道，原来的$H_{i,j}$就变成了$H_{d, i,j}$，这就要求我们的参数还要扩展一个维度。我们的公式就变成了：

$$H_{d,i,j} = \sum_c\sum_{a=-\Delta}^{\Delta}\sum_{b=-\Delta}^{\Delta} W_{d,c,a,b}X_{i + a, j + b} + B_{d}$$

# 卷积的计算

在卷积的计算中最重要的一点，就是计算好卷积层的输出尺寸。

- 输入图像的shape为$(n_h,n_w)$
- 扩边的大小为$(p_h,p_w)$
- 卷积核的大小为$(k_h,k_w)$
- 步长为$(s_h,s_w)$

卷积的输出的shape为
$$(\frac{n_h + p_h - k_h}{s_h} + 1, \frac{n_w + p_w - k_w}{s_w} + 1)$$

<img src='./images/conv_output_shape.jpg' width=600>

In [57]:
def conv2d(X, kernel, padding, strides, bias = 0):
    nh, nw = X.shape
    kh, kw = kernel.shape
    ph, pw = padding
    sh, sw = strides
    X_p = torch.zeros(nh + ph, nw + pw)# padding
    X_p[ph//2:nh+ph//2, pw//2:nw+pw//2] = X 
    Y = torch.zeros((X_p.shape[0] - kh + sh) // sh, (X_p.shape[1] - kw + sw) // sw)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = torch.sum(X_p[i*sh:i*sh+kh,j*sw:j*sw+kw].mul(kernel)) + bias
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
conv2d(X, K, (0,0), (1,1))

tensor([[19., 25.],
        [37., 43.]])

In [62]:
# 支持X有多个Channel
def conv2d_multi_input_channel(X, kernel, padding, strides, bias = 0):
    return torch.stack([conv2d(XC, kc, padding, strides, bias) for XC, kc in zip(X, kernel)]).sum(axis=0)
XC = torch.stack([X, X, X])
KC = torch.stack([K, K, K])
conv2d_multi_input_channel(XC, KC, (0,0), (1,1))

tensor([[ 57.,  75.],
        [111., 129.]])

In [66]:
# 支持多个输出通道
def conv2d_multi_iutput_and_output_channel(XC, kernel, padding, strides, bias = 0):
    return torch.stack([conv2d_multi_input_channel(XC, k, padding, strides, bias) for k in kernel])

MKC = torch.stack([KC, KC])
conv2d_multi_iutput_and_output_channel(XC, MKC, (0,0), (1,1))

tensor([[[ 57.,  75.],
         [111., 129.]],

        [[ 57.,  75.],
         [111., 129.]]])

进一步，我们用上面的卷积运算函数，构造我们卷积层

In [18]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        # Parameter类型，可以当前Tensor类型来直接使用
        return conv2d(x, self.weight, self.bias)
conv2d_layer = Conv2D((2,2))

In [16]:
conv2d_layer(X)

tensor([[-4.6859, -5.9690],
        [-8.5354, -9.8185]], grad_fn=<CopySlices>)

# featuremap感受野的问题

如果我们的网络结构都是卷积层，而且每个卷积层的卷积核的大小都是$k\times k$，则对于第l层的featuremap，其中一个位置，对应输入数据上的感受野的大小为：
$$(l * (k - 1) + 1)\times(l * (k - 1) + 1)$$

比如我们的卷积核都是$3\times 3$，一共有10层卷积，则第10层的一个输出，对应输入层的大小为$10\times(3-1) + 1=21$

# 1x1卷积

`1x1`的卷积听起来很奇怪，因为之前卷积都是考虑到一个邻域的信息的，而`1x1`的领域，只在多个通道之间做融合。在实际的网络设计中也十分的有用。而且`1x1`的卷积，其实可以看为是全连接。

In [68]:
def conv_2d_1x1(X, kernel):
    nc,nh,nw = X.shape
    kn = kernel.shape[0]
    X = X.reshape((nc, nh * nw))
    kernel = kernel.reshape((kn, nc))
    return (kernel.mul(X)).reshape(kn, nh, nw)

# 计算量的问题

对下面参数的卷积运算

- 输入图像的shape为$(n_c,n_h,n_w)$
- 扩边的大小为$(p_h,p_w)$
- 卷积核的形状为$(k_n, k_c, k_h,k_w)$
- 步长为$(s_h,s_w)$

它的输出的形状为$(o_c, o_h, o_w)$

$$(o_c, o_h, o_w) = (k_n, \frac{n_h + p_h - k_h}{s_h} + 1, \frac{n_w + p_w - k_w}{s_w} + 1)$$

总共执行的乘法的次数为：

$$o_c\cdot o_h\cdot o_w\cdot k_c \cdot k_h \cdot k_w$$

总共执行的加法次数为：

$$o_c\cdot o_h\cdot o_w\cdot (k_c \cdot k_h \cdot k_w + (k_c -1) + 1)$$